In [1]:
from datetime import datetime, timedelta

# Package import
import praw
import pprint

# Local Import
import comment_processing
import database_things
import usernamevalidator

from datetime import datetime

import pymongo

import re
import string

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [2]:
client = pymongo.MongoClient()
db = client.reddit

collection = db['word_analysis']
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

In [4]:
def preprocess(s, lowercase=False):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
# print(preprocess())

In [5]:
reddit = praw.Reddit(client_id='JLn24zAHmQ3iPg',
                     client_secret='KnsAYz4tTBdSpX7LMB1KbjG998M',
                     user_agent='windows:com.top_word_finder.XD:v0.2 by /u/waitingforcracks',
                     username='waitingforcracks',
                     password='1011sailboat')

print('Read only:', reddit.read_only)  # Check if read_only, should be false
print('Ok')

Read only: False
Ok


In [ ]:
try:
    print(reddit.redditor('spez').id)
    print('exists')
except prawcore.exceptions.NotFound:
    print('Not Found')

In [ ]:
def username_validator(username, db=db):
    
    usernames_collection = db['valid_usernames']
    print(usernames_collection)
    
    valid = usernames_collection.find_one({'username': username})
    pprint(valid)
    
    if valid is None:
        print('Not Found')
    else:
        print('ok')

In [ ]:
# database

client = pymongo.MongoClient()
db = client['reddit']
collection = db['word_analysis']

In [ ]:
record = collection.find_one({'redditor': 'dsdsd'})

pprint.pprint(record)

In [6]:
def word_finder(comments):
    # takes a reddit.reddditor(username).comments.top and returns a dict and number of comments
    comment_count = 0
    word_dict = {}
    for comment in comments:
        comment_count += 1
        # print(comment_count)
        print(comment)
        comment_tokens = preprocess(comment.body)

        for wordy in comment_tokens:
            wordy = wordy.casefold()
            if wordy not in string.punctuation:
                if wordy not in stopwords.words('english'):
                    if wordy in word_dict:
                        word_dict[wordy] += 1
                    else:
                        word_dict[wordy] = 1

    return word_dict, comment_count


In [9]:
def comments_all(redditor, sorty):
    if sorty == 'top':
        comments = reddit.redditor(redditor).comments.top(limit=None)
    elif sorty == 'new':
        comments = reddit.redditor(redditor).comments.new(limit=None)
    elif sorty == 'controversial':
        comments = reddit.redditor(redditor).comments.controversial(limit=None)
    elif sorty == 'hot':
        comments = reddit.redditor(redditor).comments.hot(limit=None)
    else:
        print(sorty)
        raise Exception('Unknown Sorting')

    return comments

In [10]:
def data_fetch(redditor, sorty):
    comment_contructor = comments_all(redditor=redditor, sorty=sorty)

    word_dict, comment_count = comment_processing.word_finder(comment_contructor)

    word = max(word_dict, key=word_dict.get)
    word_count = word_dict[word]

    return word, word_count, comment_count

In [11]:
data_fetch('spez', 'top')

d992fwq
d992soe
damgw92
d992zfl
d993fog
d3uulqk
csz1fte
damgif0
d4bun0e
damfr71
cszv2lg
csz2dsx
cszvwgm
d994hlf
d9929mu
d9942e3
csz29ee
cszucsl
damftsk
cszuw6k
d47z5l9
d99346a
cszv6yi
cszukn4
cszwvz3
csz1vdq
d1kpabj
csz2mqs
ctk8mg7
damgmgw
dfs940o
damhcpv
cszwbma
csz2lzu
ctk9lz3
cszv42m
csztsfm
d3uvxe1
dcwaqn1
cszuof8
d994zha
d993ea3
cszv5ca
cszuqyx
ctsqobs
d992ujb
csz1g9d
ct5uk86
d9ftvhq
d1koeqt
dfs9g3y
d3uv1nz
csz2jfe
dfsaabi
damh7q2
dcwcilx
damg9kn
cszw11n
cszu3cv
d993osx
d993qii
ct5rm74
ct5r5w7
d6c1zpy
d3uu6eg
cszctiu
czfdvsw
cszwgi0
damhm8o
cw35ukg
d1kpn4k
cx74hdv
d3uucfq
damff2k
ct5vfj1
c5c8v56
damj0ra
d993mos
d3uwt30
czfic7q
dcwb34i
d4bv47n
d4bu4au
d3uw9aj
cw33sk5
d992doa
cszw3cy
cw346hw
ctsqsj8
d9929yb
d993i70
c0c95xr
cszwaqv
d6c30zo
cszx5hr
ct5sxbm
dfs9kvt
d3uuhmr
ctk9qnl
cszvueb
ct5umqt
damg7dw
dami1gy
d47y97r
d4btcxc
damk30f
ct5v522
cszwd4d
d99421p
csztvag
d992xnc
cx74t7f
csz1pif
ctk8w1v
ct5tbcs
cszwuob
cszvpfy
d993t8t
d9943rg
d3uv2sw
csz338q
d3uxivs
cszuze0
ct5sip8
ctk9g7d


('reddit', 211, 994)